# **Imports 📢**

In [64]:
!pip install torchmetrics

In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torchmetrics import Accuracy
from tqdm import tqdm

# **Dataset 🗂️**

## Read Dataset

In [66]:
data_frame_train = pd.read_csv('/content/train.csv')
date_frame_test = pd.read_csv('/content/test.csv')

In [67]:
x_train = data_frame_train.drop('price_range', axis=1).values
y_train = data_frame_train['price_range'].values

## Split

In [68]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, train_size=0.7, random_state=42)

## Preprocess

In [69]:
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

x_valid = torch.FloatTensor(x_valid)
y_valid = torch.LongTensor(y_valid)

mu = x_train.mean(dim=0)
std = x_train.std(dim=0)

x_train = (x_train - mu) / std
x_valid = (x_valid - mu) / std

## Dataloader

In [70]:
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, 20, True)

valid_dataset = TensorDataset(x_valid, y_valid)
valid_loader = DataLoader(valid_dataset, batch_size=200)

# **Model 🧠**

In [71]:
num_feats = 20
num_class = 4
h1 = 64
h2 = 32

model = nn.Sequential(nn.Linear(num_feats, h1),
                      nn.ReLU(),
                      # nn.Dropout(0.5),
                      nn.Linear(h1, h2),
                      nn.ReLU(),
                      nn.Linear(h2, num_class))

In [ ]:
model[0].bias



## *   torch.nn
## *   Dropout



# **Loss & Optimizer ⚖️**

In [73]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01,
                      momentum=0.9, nesterov=True,
                      weight_decay=1e-4)

# **Device ⚙️**

In [74]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# **Utils 🧰**

In [75]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# **Functions** 🧮

In [76]:
def train_one_epoch(model, train_loader, loss_fn, optimizer, epoch=None):
  model.train()
  loss_train = AverageMeter()
  acc_train = Accuracy(task='multiclass', num_classes=4).to(device)
  with tqdm(train_loader, unit='batch') as tepoch:
    for inputs, targets in tepoch:
      if epoch is not None:
        tepoch.set_description(f'Epoch {epoch}')
      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = model(inputs)
      loss = loss_fn(outputs, targets)

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      loss_train.update(loss.item())
      acc_train(outputs, targets.int())

      tepoch.set_postfix(loss=loss_train.avg,
                         accuracy=100.*acc_train.compute().item())
  return model, loss_train.avg, acc_train.compute().item()

In [77]:
def evaluate(model, valid_loader, loss_fn):
  model.eval()
  with torch.no_grad():
    loss_valid = AverageMeter()
    acc_valid = Accuracy(task='multiclass', num_classes=4).to(device)
    for i, (inputs, targets) in enumerate(valid_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = model(inputs)
      loss = loss_fn(outputs, targets)

      loss_valid.update(loss.item())
      acc_valid(outputs, targets.int())

  return loss_valid.avg, acc_valid.compute()

# **Train**

In [ ]:
num_epochs = 100

loss_train_hist = []
loss_valid_hist = []

acc_train_hist = []
acc_valid_hist = []

for epoch in range(num_epochs):
  model, loss_train, acc_train = train_one_epoch(model,
                                                 train_loader,
                                                 loss_fn,
                                                 optimizer,
                                                 epoch)

  loss_valid, acc_valid = evaluate(model,
                                   valid_loader,
                                   loss_fn)

  loss_train_hist.append(loss_train)
  loss_valid_hist.append(loss_valid)

  acc_train_hist.append(acc_train)
  acc_valid_hist.append(acc_valid)

  print(f'Valid: Loss={loss_valid:.4}, Accuracy={acc_valid:.4}\n')

# **Plot**

### Loss

In [ ]:
plt.plot(range(num_epochs), loss_train_hist, 'r-', label='Train')
plt.plot(range(num_epochs), loss_valid_hist, 'b-', label='Validation')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

### Accuracy

In [ ]:
plt.plot(range(num_epochs), acc_train_hist, 'r-', label='Train')
plt.plot(range(num_epochs), acc_valid_hist, 'b-', label='Validation')

plt.xlabel('Epoch')
plt.ylabel('Acc')
plt.grid(True)
plt.legend()

In [81]:
state_dict = model.state_dict()

In [82]:
torch.save(state_dict, 'weights.pt')

In [83]:
wb = torch.load('weights.pt')
model.load_state_dict(wb)

# **Efficient way for set hyperparams 🔨**

In [84]:
model = nn.Sequential(nn.Linear(num_feats, h1),
                      nn.ReLU(),
                      nn.Linear(h1, h2),
                      nn.ReLU(),
                      nn.Linear(h2, num_class))

optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

## Step 1: check forward path

Calculate loss for one batch

In [93]:
x_batch, y_batch = next(iter(train_loader))
yp = model(x_batch)
loss_fn(yp, y_batch)

torch.Size([20, 20])

## Step 2: check backward path

Select 5 random batches and overfit the model

In [86]:
mini_train_dataset, _ = random_split(train_dataset, (100, len(train_dataset)-100))
mini_loader = DataLoader(mini_train_dataset, 20, shuffle=True)

In [60]:
for epoch in range(500):
    model, _, _ = train_one_epoch(model, mini_loader, loss_fn, optimizer, epoch)

## Step 3: select best lr

Train all data for one epoch

In [61]:
model = nn.Sequential(nn.Linear(num_feats, h1),
                      nn.ReLU(),
                      nn.Linear(h1, h2),
                      nn.ReLU(),
                      nn.Linear(h2, num_class))

optimizer = optim.SGD(model.parameters(), lr=..., weight_decay=...)

In [ ]:
for lr in [0.1, 0.01, 0.001, 0.0001]:
  model = nn.Sequential(nn.Linear(num_feats, h1),
                      nn.ReLU(),
                      nn.Linear(h1, h2),
                      nn.ReLU(),
                      nn.Linear(h2, num_class))

  optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=1e-4)
  print(f'LR={lr}')
  for epoch in range(5):
    model, _, _ = train_one_epoch(model, train_loader, loss_fn, optimizer, epoch)
  print()

## Step 4: small grid (optional)

Create a small grid based on the WD and the best LR



In [ ]:
model = nn.Sequential(nn.Linear(num_feats, h1),
                      nn.ReLU(),
                      nn.Linear(h1, h2),
                      nn.ReLU(),
                      nn.Linear(h2, num_class))

optimizer = optim.SGD(model.parameters(), lr=..., weight_decay=...)

In [ ]:
for lr in [0.1, 0.15, 0.2, 0.25, 0.3]:
  for wd in [0., 1e-4, 1e-5, 1e-6]:
    model = nn.Sequential(nn.Linear(num_feats, h1),
                      nn.ReLU(),
                      nn.Linear(h1, h2),
                      nn.ReLU(),
                      nn.Linear(h2, num_class))

    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd)

    print(f'LR={lr}, WD={wd}')
    for epoch in range(5):
      model, _, _ = train_one_epoch(model,
                                    train_loader,
                                    loss_fn,
                                    optimizer,
                                    epoch)
    print()
    # torch.save(...)

### Pretty table

In [ ]:
from prettytable import PrettyTable

l = [["Hassan", 21, "LUMS"], ["Ali", 22, "FAST"], ["Ahmed", 23, "UET"]]
table = PrettyTable(['Name', 'Age', 'University'])

for rec in l:
    table.add_row(rec)

print(table)

## Step 5: train more epochs

In [ ]:
model = nn.Sequential(nn.Linear(num_feats, h1),
                      nn.ReLU(),
                      nn.Linear(h1, h2),
                      nn.ReLU(),
                      nn.Linear(h2, num_class))

optimizer = optim.SGD(model.parameters(), lr=0.15, weight_decay=1e-5)

In [ ]:
num_epochs = 20

In [ ]:
best_valid_loss = torch.inf

In [ ]:
for epoch in range(num_epochs):
  # Train
  model, loss_train, acc_train = train_one_epoch(model,
                                                 train_loader,
                                                 loss_fn,
                                                 optimizer,
                                                 epoch)
  # Validation
  loss_valid, acc_valid = evaluate(model,
                                     valid_loader,
                                     loss_fn)

  if loss_valid < best_valid_loss:
    torch.save(model, 'model.pt')
    best_valid_loss = loss_valid

  print(f'Valid: Loss = {loss_valid:.4}, Acc = {acc_valid:.4}\n')